In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from IPython.display import HTML
from matplotlib.dates import DateFormatter
from datetime import datetime
from datetime import timedelta

import plotly.offline as pyo
import plotly.graph_objs as go 
from plotly.subplots import make_subplots

In [2]:
# setting up disply options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Our world in data - import data
url = 'https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv?raw=true'

dfVaccination = pd.read_csv(url, 
                            encoding = 'utf-8', 
                            parse_dates=['date'],
                            sep = ',')
dfVaccination.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [4]:
dfVaccination.dtypes

iso_code                                         object
continent                                        object
location                                         object
date                                     datetime64[ns]
total_cases                                     float64
new_cases                                       float64
new_cases_smoothed                              float64
total_deaths                                    float64
new_deaths                                      float64
new_deaths_smoothed                             float64
total_cases_per_million                         float64
new_cases_per_million                           float64
new_cases_smoothed_per_million                  float64
total_deaths_per_million                        float64
new_deaths_per_million                          float64
new_deaths_smoothed_per_million                 float64
reproduction_rate                               float64
icu_patients                                    

In [5]:
# drop continents from data frame
countries = ['South America', 'North America', 'World', 'Oceania','Europe', 'International', 'Africa', 'Asia', 'European Union']
i = dfVaccination[dfVaccination.location.isin(countries)].index

In [6]:
dfVaccination.drop(i, inplace=True)

In [7]:
# Checking the max avaiable date
dfVaccination.date.max()

Timestamp('2021-04-05 00:00:00')

In [8]:
# Creating support columns for the dataframe
dfVaccination['new_cases_per_million_rollmean7'] = round(dfVaccination['new_cases_per_million'].rolling(7, center=True).mean(),2) # Rolling mean 7 days
dfVaccination['new_deaths_per_million_rollmean7'] = round(dfVaccination['new_deaths_per_million'].rolling(7, center=True).mean(),2) # Rolling mean 7 days
dfVaccination['total_tests_per_million'] = round((dfVaccination['total_tests']/dfVaccination['population'])*1000000,2)
dfVaccination['people_fully_vaccinated_per_million'] = round((dfVaccination['people_fully_vaccinated']/dfVaccination['population'])*1000000,2)

dfVaccination['people_fully_vaccinated_perc'] = round((dfVaccination['people_fully_vaccinated']/ dfVaccination['population'])*100,2)


In [9]:
dfVaccination.dtypes

iso_code                                         object
continent                                        object
location                                         object
date                                     datetime64[ns]
total_cases                                     float64
new_cases                                       float64
new_cases_smoothed                              float64
total_deaths                                    float64
new_deaths                                      float64
new_deaths_smoothed                             float64
total_cases_per_million                         float64
new_cases_per_million                           float64
new_cases_smoothed_per_million                  float64
total_deaths_per_million                        float64
new_deaths_per_million                          float64
new_deaths_smoothed_per_million                 float64
reproduction_rate                               float64
icu_patients                                    

In [10]:
# Plotagem de evolução de casos e média móvel centrada dos últimos
# 7 dias
def plotagem_media_movel(dfPlot:            pd.DataFrame,
                         datecolumn:        str,
                         qtdcolumn:         str,
                         smoothmeancolumn:  str,
                         title:             str):
  #-----------------------------------------------------------#
  # dfPlot: DataFrame com casos para o plot                   #
  # datecolumn: Nome da Coluna de Data do Daframe             #
  # qtdcolumn: Nome da Coluna as quantidades de casos         #
  # smoothmeancolumn: Coluna com a média móvel  de 7 dias     #
  # titulo: Título do Gráfico                                 #
  # Retorna a figura já plotada                               #
  #-----------------------------------------------------------#
  layout = go.Layout(
    xaxis_title_text = 'The side handlebars act as filters',
    template = 'plotly_white',
    title = { 'text': f'{title}<br>**Interactive Graph**', 
            'xanchor': 'center',
            'yanchor': 'top', 'y': 0.95, 'x': 0.45, 'font': dict(size=20)
          },
    xaxis=dict(
        title='Date',
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        title='Cases - Daily '
    ),
    hovermode='closest',
  )

  trace1 = go.Bar(
      x=dfPlot[f'{datecolumn}'],
      y=dfPlot[f'{qtdcolumn}'],
      name='Daily'
  )

  trace2 = go.Scatter(
      x=dfPlot[f'{datecolumn}'],
      y=dfPlot[f'{smoothmeancolumn}'],
      mode='lines',
          marker=dict(
              color='Red',
              size=120,
              line=dict(
                  color='Red',
                  width=20
              )),
      name='Rolling Mean - 7 days'
  )

  data=[trace1, trace2]

  figure = go.Figure(data=data, layout=layout)

  figure['layout'].update(
      legend=dict(
          orientation="h",
          x=0,
          y=1.0
      ),
      xaxis=dict(
          rangeslider=dict(
              visible = True
          ),
          type='date',
      ),
  )

  figure.show()

In [16]:
to_plot = dfVaccination.loc[dfVaccination.location=='Brazil']

In [99]:
country1 = dfVaccination.loc[dfVaccination.location=='Australia']
country2 = dfVaccination.loc[dfVaccination.location=='United States']


In [20]:

country1['new_deaths'].sum()/country1['new_cases'].sum()*100





today = pd.to_datetime('today')

last_month = today - pd.DateOffset(months=1)








last_month_cases = country1.loc[country1['date'] < last_month, 'new_cases'].sum()

this_month_cases = country1.loc[country1['date'] >= last_month, 'new_cases'].sum()

growth = -100 - round(((this_month_cases-last_month_cases)/last_month_cases)*100,2)
Mortality_Rate1         = country1['new_deaths'].sum()/country1['new_cases'].sum()*100

growth =  round(abs(1+(this_month_cases-last_month_cases)/last_month_cases)*100,2)

print(growth)
print(Mortality_Rate1)
#print(last_month_deaths)
#print(this_month_deaths)



18.97
2.556955603602723


In [21]:
plotagem_media_movel(to_plot, 
                     'date', 
                     'new_cases', 
                     'new_cases_per_million_rollmean7',	
                     'Cases Mean')

In [22]:
  Vaccination_Start_Date1 = country1.loc[~country1['new_vaccinations'].isnull()].iloc[0]['date']
  Vaccination_Start_Date1 = Vaccination_Start_Date1.strftime("%m/%d/%y")
  
  Vaccination_Start_Date1

'01/17/21'

In [23]:
Vaccination_Start_Date1

'01/17/21'

In [144]:
# Função para plotar painel de Casos de COVID-19, óbitos por COVID-19
# e Percentual de Isolamento.

def COVID_Cases_per_million(dfCountry1: pd.DataFrame, 
                            dfCountry2: pd.DataFrame,
                            cTitleCountry1,
                            cTitleCountry2,
                            titulo: str):
  #-----------------------------------------------------------#
  # dfCountry1: Country 1 DataFrame                           #
  # dfCountry2: Country 2 DataFrame                           #
  # cTitleCountry1: selected country1                         #
  # cTitleCountry1: selected country2                         #
  # titulo: Graph Title                                       #
  # Returns: Ploted Graph                                     #
  #-----------------------------------------------------------#

  fig = make_subplots(rows=3, 
                      cols=2, 
                      vertical_spacing=0.15,
                      horizontal_spacing=0.08,
                      specs=[[{"type": "scatter"}, {"type": "scatter"}],
                             [{"type": "scatter"}, {"type": "scatter"}],                          
                             [{"type": "table"}, {"type": "table"}]],
                      subplot_titles=(f"<b>Daily cases per million in {cTitleCountry1}</b>",
                                      f"<b>Daily cases per million in {cTitleCountry2}</b>",
                                      f"<b>Testing and Vaccination Evolution per million in {cTitleCountry1}</b>",     
                                      f"<b>Testing and Vaccination Evolution per million in {cTitleCountry2}</b>",
                                      f"<b>Demographics of {cTitleCountry1}</b>",
                                      f"<b>Demographics of {cTitleCountry2}</b>"))
                                              
  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['new_cases_per_million'],
                marker=dict(
                            color='deepskyblue',
                            size=120
          ),
      name=f"Daily cases per million in {cTitleCountry1}",
      hovertemplate=
        f"<b>{cTitleCountry1}'s new cases per million</b><br><br>" +
        "Date: %{x}<br>" +
        "New cases per million: %{y:,.}<br>" +
        "<extra></extra>"),
      row=1, col=1
  )

  fig.add_trace(
    go.Scatter(
    x=dfCountry1['date'],
    y=dfCountry1['new_cases_per_million_rollmean7'],
    mode='lines',
        marker=dict(
            color='Red',
            size=120,
            line=dict(
                color='Red',
                width=20
            )),
    name=f"{cTitleCountry1}'s 7 day rolling mean",
    hovertemplate=
        f"<b>{cTitleCountry1}'s 7 day rolling mean</b><br><br>" +
        "Date: %{x}<br>" +
        "New cases: %{y:,.}<br>" +
        "<extra></extra>"),
    row=1, col=1)

  fig.add_trace(
      go.Scatter(x=dfCountry2['date'], 
                y=dfCountry2['new_cases_per_million'],
                marker=dict(
                            color='gold',
                            size=120,
          ), 
      name=f"Daily Cases in {cTitleCountry2}",
      hovertemplate=
        f"<b>{cTitleCountry2}'s new cases per million </b><br><br>" +
        "Date: %{x}<br>" +
        "New cases per million: %{y:,.}<br>" +
        "<extra></extra>"),      
      row=1, col=2
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['new_cases_per_million_rollmean7'],
      mode='lines',
          marker=dict(
              color='Red',
              size=120,
              line=dict(
                  color='Red',
                  width=20
              )),
      name=f"{cTitleCountry2}'s 7 day rolling mean",
      hovertemplate=
        f"<b>{cTitleCountry2}'s 7 day rolling mean</b><br><br>" +
        "Date: %{x}<br>" +
        "New cases per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=1, col=2)
    
  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['people_fully_vaccinated_per_million'],
                marker=dict(color='Purple',
                            size=160,
                            line=dict(
                            color='Purple',
                            width=40)), 
      name="People Fully Vaccinated per Million",
      hovertemplate=
        f"<b>{cTitleCountry1}'s fully vaccinated per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>"),   
      row=2, col=1
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry1['date'],
      y=dfCountry1['new_vaccinations_smoothed_per_million'],
      mode='lines',
          marker=dict(
              color='darkorchid',
              size=160,
              line=dict(
                  color='darkorchid',
                  width=40
              )),
      name='New Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry2}'s vaccinated per million<br>(at least one dose) </b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=1)

  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['total_tests_per_million'],
                marker=dict(
                            color='Green',
                            size=180,
                            line=dict(
                            color='Green',
                            width=40)),
      name="People Tested per Million", 
      hovertemplate=
        f"<b>{cTitleCountry1}'s tested per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Tested per million: %{y:,.}<br>" +
        "<extra></extra>"),
      row=2, col=1
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['new_vaccinations_smoothed_per_million'],
      mode='lines',
          marker=dict(
              color='darkorchid',
              size=160,
              line=dict(
                  color='darkorchid',
                  width=40
              )),
      name='People Fully Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry2}'s vaccinated per million<br>(at least one dose) </b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['people_fully_vaccinated_per_million'],
      mode='lines',
          marker=dict(
              color='Purple',
              size=160,
              line=dict(
                  color='Purple',
                  width=40
              )),
      name='People Fully Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry2}'s fully vaccinated per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['total_tests_per_million'],
      mode='lines',
          marker=dict(
              color='Green',
              size=160,
              line=dict(
                  color='Green',
                  width=40
              )),
      name='Tested per Million',
      hovertemplate=
        f"<b>{cTitleCountry2}'s tested per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Tested per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)  

  # Table Calculations for the first country
  today = pd.to_datetime('today')
  last_month = today - pd.DateOffset(months=1)
  last_month_cases1 = dfCountry1.loc[dfCountry1['date'] < last_month, 'new_cases'].sum()
  this_month_cases1 = dfCountry1.loc[dfCountry1['date'] >= last_month, 'new_cases'].sum()
  Population1             = dfCountry1['population'].max()
  Perc_Fully_Vaccinated1  = round((dfCountry1['people_fully_vaccinated'].max()/ 
                            dfCountry1['population'].max())*100,2)
                            
  Vaccination_Start_Date1 = country1.loc[~country1['new_vaccinations'].isnull()].iloc[0]['date']

  Cases_Total1            = int(dfCountry1['total_cases'].max())
  Deaths_Total1           = int(dfCountry1['total_deaths'].max())
  Mortality_Rate1         = round(dfCountry1['new_deaths'].sum()/dfCountry1['new_cases'].sum()*100,2)
  Growth_Rate1            = round(abs(1+(this_month_cases1-last_month_cases1)/last_month_cases1)*100,2)
  cells1 = []
  cells1.append(round((Population1/1000000),1))
  cells1.append(f'{Perc_Fully_Vaccinated1}%')

  cells1.append(Vaccination_Start_Date1.strftime("%m/%d/%y"))
  cells1.append("{:,}".format(Cases_Total1))
  cells1.append("{:,}".format(Deaths_Total1))
  cells1.append(f'{Mortality_Rate1}%')
  cells1.append(f'{Growth_Rate1}%')      

  fig.add_trace(
      go.Table(
          header=dict(
              values=["<b>Population (Millions)</b>", "<b>Perc. of Population fully vaccineted</b>","<b>Vaccination started on (MM/DD/YY)</b>", "<b>Cases Total</b>", "<b>Deaths Total</b>", "<b>Mortality Rate</b>", "<b>Last Month Growth Rate</b>"],
              font=dict(size=10),
              align="center"
          ),
          cells=dict(
              values=cells1,
              align = "center")
      ),
      row=3, col=1
  )          

  # Table Calculations for the second country
  today = pd.to_datetime('today')
  last_month = today - pd.DateOffset(months=1)
  last_month_cases2 = dfCountry2.loc[dfCountry2['date'] < last_month, 'new_cases'].sum()
  this_month_cases2 = dfCountry2.loc[dfCountry2['date'] >= last_month, 'new_cases'].sum()
  Population2             = dfCountry2['population'].max()
  Perc_Fully_Vaccinated2  = round((dfCountry2['people_fully_vaccinated'].max()/ 
                            dfCountry2['population'].max())*100,2)
                            
  Vaccination_Start_Date2 = dfCountry2.loc[~dfCountry2['new_vaccinations'].isnull()].iloc[0]['date']
  Cases_Total2            = int(dfCountry2['total_cases'].max())
  Deaths_Total2           = int(dfCountry2['total_deaths'].max())
  Mortality_Rate2         = round(dfCountry2['new_deaths'].sum()/dfCountry2['new_cases'].sum()*200,2)
  Growth_Rate2            = round(abs(1+(this_month_cases2-last_month_cases2)/last_month_cases2)*200,2)
  cells2 = []
  cells2.append(round((Population2/1000000),1))
  cells2.append(f'{Perc_Fully_Vaccinated2}%')
  cells2.append(Vaccination_Start_Date2.strftime("%m/%d/%y"))
  cells2.append("{:,}".format(Cases_Total2))
  cells2.append("{:,}".format(Deaths_Total2))
  cells2.append(f'{Mortality_Rate2}%')
  cells2.append(f'{Growth_Rate2}%')

  fig.add_trace(
      go.Table(
          header=dict(
              values=["<b>Population (Millions)</b>", "<b>Perc of Population fully vaccineted</b>","<b>Vaccination started on (MM/DD/YY)</b>", "<b>Cases Total</b>", "<b>Deaths Total</b>", "<b>Mortality Rate</b>", "<b>Last Month Growth Rate</b>"],
              font=dict(size=10),
              align="center"
          ),

          cells=dict(
              values=cells2,
              align = "center")
      ),
      row=3, col=2
  ) 

  data_date = dfCountry1.date.max().strftime("%m/%d/%Y")
  plot1 = ''
  plot2 = ''

  if dfCountry1['total_tests_per_million'].isnull().values.sum():
     plot1 = f'** No all test data avaiable for {cTitleCountry1}'
  elif dfCountry2['total_tests_per_million'].isnull().values.sum():
     plot2 = f'** No all test data avaiable for {cTitleCountry2}'

  fig.update_layout(
      template = 'plotly_white',
      title = { 'text': f'{titulo}<br>**Interative Graph**',
                'font': dict(size=20),
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
      showlegend=False,        
      yaxis = {  'categoryorder': 'total descending'},
      yaxis3 = { 'categoryorder': 'total descending'},              
      autosize=False,
      width=1280,
      height=900,     
      margin=dict(
          l=50,
          r=50,
          b=10,
          t=110,
          pad=2
      )
  )

  fig.add_annotation(text=f'Source: https://ourworldindata.org/ <br> * Data available until: {data_date} <br> {plot1} <br> {plot2}',
                  xref="paper", yref="paper",
                  x=0.9999, y=0.001, showarrow=False)

  if not plot1 is None: 
    fig.add_annotation(text=f'<b>{plot1}</b>',
                    xref="paper", yref="paper",
                    x=0.03, y=0.33, showarrow=False)                        

  if not plot2 is None: 
    fig.add_annotation(text=f'<b>{plot2}</b>',
                    xref="paper", yref="paper",
                    x=0.77, y=0.33, showarrow=False)               

  fig.show()
  return 

In [89]:
country2['total_tests_per_million'].isnull().unique()

array([ True, False])

In [87]:
country2['total_tests_per_million'].isnull().values.any()

True

In [108]:
country1.people_fully_vaccinated.isnull().unique()

array([ True])

In [106]:
country1.tail()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,new_cases_per_million_rollmean7,new_deaths_per_million_rollmean7,total_tests_per_million,people_fully_vaccinated_per_million,people_fully_vaccinated_perc
4585,AUS,Oceania,Australia,2021-04-01,29333.0,11.0,13.429,909.0,0.0,0.0,1150.319,0.431,0.527,35.647,0.0,0.0,1.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85666.0,15698886.0,615.645,3.359,52043.0,2.041,0.0,3875.4,tests performed,744328.0,NaN,NaN,73979.0,47988.0,2.92,NaN,NaN,1882.0,46.76,25499881.0,3.202,37.9,15.504,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44,0.944,0.45,0.0,615645.46,NaN,NaN
4586,AUS,Oceania,Australia,2021-04-02,29341.0,8.0,12.714,909.0,0.0,0.0,1150.633,0.314,0.499,35.647,0.0,0.0,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54010.0,2.118,0.0,4248.1,tests performed,823611.0,NaN,NaN,79283.0,51922.0,3.23,NaN,NaN,2036.0,46.76,25499881.0,3.202,37.9,15.504,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44,0.944,0.39,0.0,NaN,NaN,NaN
4587,AUS,Oceania,Australia,2021-04-03,29348.0,7.0,12.714,909.0,0.0,0.0,1150.907,0.275,0.499,35.647,0.0,0.0,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55977.0,2.195,0.0,4402.8,tests performed,828416.0,NaN,NaN,4805.0,45993.0,3.25,NaN,NaN,1804.0,46.76,25499881.0,3.202,37.9,15.504,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44,0.944,0.37,NaN,NaN,NaN,NaN
4588,AUS,Oceania,Australia,2021-04-04,29357.0,9.0,11.571,909.0,0.0,0.0,1151.260,0.353,0.454,35.647,0.0,0.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57944.0,2.272,0.0,5007.7,tests performed,843182.0,NaN,NaN,14766.0,47380.0,3.31,NaN,NaN,1858.0,46.76,25499881.0,3.202,37.9,15.504,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44,0.944,0.27,NaN,NaN,NaN,NaN
4589,AUS,Oceania,Australia,2021-04-05,29365.0,8.0,9.857,909.0,0.0,0.0,1151.574,0.314,0.387,35.647,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15892476.0,623.237,NaN,58417.0,2.291,0.0,5926.4,tests performed,844309.0,NaN,NaN,1127.0,43221.0,3.31,NaN,NaN,1695.0,NaN,25499881.0,3.202,37.9,15.504,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44,0.944,0.23,NaN,623237.26,NaN,NaN


In [145]:
COVID_Cases_per_million(country1, 
                        country2,
                        'Brazil',
                        'United States of America',
                        'COVID-19 Country Comparison')